# Homework: The Dawn of Neural Machine Translation with Seq2Seq

## Robert Clay Harris: jbm2rt

## Part 1: Historical Context and Motivation

Before the rise of deep learning, machine translation (MT) was dominated by **Statistical Machine Translation (SMT)**. SMT systems were complex engineering feats, relying on statistical models to translate phrases piece-by-piece and then reassembling them using intricate rules.

In 2014, a seminal paper changed the landscape: **"Sequence to Sequence Learning with Neural Networks"** by Sutskever, Vinyals, and Le. They proposed an elegant, end-to-end neural architecture.

### The Core Idea

The core idea is remarkably simple:

1. **The Encoder**: An RNN reads the input sentence (e.g., English) one word at a time, compressing the entire meaning into a single, fixed-size vector. This is often called the **context vector** or, more poetically, a **"thought vector."**

2. **The Decoder**: Another RNN takes this "thought vector" as its starting point and generates the output sentence (e.g., French) one word at a time.

This architecture marked the beginning of **Neural Machine Translation (NMT)**. In 2016, Google Translate switched from its older SMT system to NMT. The improvement was dramatic.

> **"With this update, Google Translate is improving more in a single leap than we've seen in the last ten years combined."** – [Google Blog, 2016 ](https://blog.google/products/translate/found-translation-more-accurate-fluent-sentences-google-translate/)

The original 2014 paper used LSTMs. However, we will use **Gated Recurrent Units (GRUs)** for this assignment just to mix it up! GRUs are similar to LSTMs in that they use gates to control information flow, but their architecture is simpler (two gates vs. three, and no separate cell state). They often perform similarly to LSTMs but are slightly faster to train and easier to implement.

---

## Part 2: Key Concepts

### 2.1 Backpropagation Through Time (BPTT)

When training RNNs, we must backpropagate gradients through all time steps of the sequence. This is called **Backpropagation Through Time (BPTT)**. The gradients flow backwards through the unrolled RNN, allowing the model to learn long-term dependencies.

### 2.2 BPTT and Truncated BPTT (TBPTT)

If a sequence is very long (e.g., modeling an entire document), full BPTT consumes excessive memory because we must store the activations for every time step.

**Truncated BPTT (TBPTT)** solves this by breaking the sequence into chunks. We process a chunk, backpropagate gradients only within that chunk, and then pass the hidden state forward to the next chunk, stopping the gradient flow at the chunk boundary.

In this assignment, our sentences are short, so we will use standard BPTT.

### 2.3 The "Reversal Trick"

The 2014 paper discovered a surprising trick that significantly boosted performance: **Reverse the source sentence.**

- **Original**: [I, love, AI] → [J'aime, l'IA]
- **Reversed**: [AI, love, I] → [J'aime, l'IA]

By doing this, the first words of the output (J'aime) are very close to the corresponding words in the reversed input (I). This creates short-term dependencies, making it much easier for the optimizer to "establish communication" between the input and the output early in training.

### 2.4 Teacher Forcing

When training the decoder, if we use the model's prediction as the input for the subsequent step, an early mistake can cascade, making training unstable.

**Teacher Forcing** is a strategy where we sometimes use the actual ground truth token from the training data as the input for the next step, rather than the model's own prediction.

---

## Part 3: Setup and Data Preprocessing

We will use a dataset of English-French sentence pairs.

### 3.0 Download the Data

Run this cell in Colab to download and unzip the data:

```bash
!wget https://download.pytorch.org/tutorial/data.zip
!unzip -o data.zip
```

### 3.1 Imports and Utilities (Provided)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
# Utilities for handling variable length sequences
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

import numpy as np
import random
import math
import time
import unicodedata
import re

# Set random seeds for reproducibility
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Define special tokens
PAD_IDX = 0
SOS_IDX = 1
EOS_IDX = 2
UNK_IDX = 3

Using device: cuda


### 3.2 Vocabulary and Data Loading (Provided)

We provide the utilities to load, normalize, and filter the data. We limit the dataset size and sentence length for faster training.

In [7]:
class Lang:
    """A class to hold the vocabulary of a language."""
    def __init__(self, name):
        self.name = name
        self.word2index = {"<PAD>": PAD_IDX, "<SOS>": SOS_IDX, "<EOS>": EOS_IDX, "<UNK>": UNK_IDX}
        self.index2word = {PAD_IDX: "<PAD>", SOS_IDX: "<SOS>", EOS_IDX: "<EOS>", UNK_IDX: "<UNK>"}
        self.n_words = 4

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1

def normalizeString(s):
    s = s.lower().strip()
    # Normalize Unicode characters (e.g., remove accents)
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s.strip()

# We filter for relatively short sentences
MAX_LENGTH = 15
NUM_EXAMPLES = 15000

def prepareData(lang1, lang2):
    print("Reading lines...")
    lines = open(f'data/{lang1}-{lang2}.txt', encoding='utf-8').read().strip().split('\n')
    
    # Limit the number of examples and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines[:NUM_EXAMPLES]]

    # Filter pairs by length
    pairs = [p for p in pairs if len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH]
    
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    print(f"Trimmed to {len(pairs)} sentence pairs")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print(f"Vocabularies: {input_lang.name} ({input_lang.n_words}), {output_lang.name} ({output_lang.n_words})")
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('eng', 'fra')

Reading lines...
Trimmed to 15000 sentence pairs
Vocabularies: eng (2830), fra (5098)


### 3.3 Dataset and DataLoader (Provided)

We implement the PyTorch Dataset. This is where we apply the **Input Reversal Trick**.

We also implement a `collate_fn`. This function handles padding sequences in a batch to the same length. Crucially, it also returns the original lengths of the sequences, which we need for **Packing**.

In [8]:
class TranslationDataset(Dataset):
    def __init__(self, pairs, input_lang, output_lang, reverse_source=True):
        self.pairs = pairs
        self.input_lang = input_lang
        self.output_lang = output_lang
        self.reverse_source = reverse_source

    def __len__(self):
        return len(self.pairs)

    def indexesFromSentence(self, lang, sentence):
        return [lang.word2index.get(word, UNK_IDX) for word in sentence.split(' ')]

    def __getitem__(self, idx):
        pair = self.pairs[idx]
        src_text = pair[0]
        tgt_text = pair[1]

        src_indices = self.indexesFromSentence(self.input_lang, src_text)
        tgt_indices = self.indexesFromSentence(self.output_lang, tgt_text)

        # Apply the Reversal Trick to the source sentence
        if self.reverse_source:
            src_indices.reverse()

        # Add EOS token to both
        src_indices.append(EOS_IDX)
        tgt_indices.append(EOS_IDX)

        return torch.tensor(src_indices, dtype=torch.long), \
               torch.tensor(tgt_indices, dtype=torch.long)

# Collate function to handle padding and return lengths
def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_item, tgt_item in batch:
        src_batch.append(src_item)
        tgt_batch.append(tgt_item)
    
    # Get the lengths of the source sequences BEFORE padding
    src_lengths = torch.tensor([len(s) for s in src_batch])
    
    # Pad the sequences
    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, batch_first=True, padding_value=PAD_IDX)

    # We return the lengths as well for packing later
    return src_batch.to(device), src_lengths, tgt_batch.to(device)

# Create Datasets and DataLoaders
BATCH_SIZE = 64
dataset = TranslationDataset(pairs, input_lang, output_lang, reverse_source=True)

# Split into train and validation (90/10)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

---

## Part 4: The Seq2Seq Architecture (Implementation Tasks)

Now we implement the core components.

### Task 1: The Encoder (20 Points)

The Encoder processes the input sequence and compresses it into the context vector.

**Important: Packing Padded Sequences.** When training RNNs on batches, we must use `pack_padded_sequence`. This tells the GRU/LSTM to ignore PAD tokens. If we don't pack, the RNN processes the padding, which wastes computation and can negatively affect the final hidden state (the context vector).

**Instructions:**

1. Initialize the `nn.Embedding` and `nn.GRU` layers. Use `batch_first=True`.
2. In the forward pass, embed the input.
3. Pack the embedded sequence using `pack_padded_sequence`.
4. Pass the packed sequence through the GRU.
5. Return the final hidden state.

In [9]:
class EncoderGRU(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # Embedding
        self.embedding = nn.Embedding(input_dim, emb_dim, padding_idx=PAD_IDX)

        # GRU
        self.rnn = nn.GRU(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0.0
        )

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_lengths):
        # src: (batch, src_len); src_lengths: (batch,)
        embedded = self.dropout(self.embedding(src))  # (batch, src_len, emb_dim)

        # Pack (ignore padding)
        packed_embedded = pack_padded_sequence(
            embedded, src_lengths.cpu(), batch_first=True, enforce_sorted=False
        )

        # RNN
        packed_outputs, hidden = self.rnn(packed_embedded)  # hidden: (n_layers, batch, hid_dim)

        return hidden

### Task 2: The Decoder (20 Points)

The Decoder takes the context vector as its initial hidden state and generates the output sequence one token at a time.

**Instructions:**

1. Initialize the Embedding, GRU, and output Linear (`fc_out`) layers.
2. The forward pass accepts one token (`input`) and the previous hidden state.
3. Embed the input token (remembering to add a sequence dimension).
4. Pass the embedding and hidden state to the GRU.
5. Pass the GRU output through the linear layer to get the prediction logits.
6. Return the prediction and the new hidden state.

In [10]:
class DecoderGRU(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.n_layers = n_layers

        # Embedding
        self.embedding = nn.Embedding(output_dim, emb_dim, padding_idx=PAD_IDX)

        # GRU (must match encoder hid_dim)
        self.rnn = nn.GRU(
            input_size=emb_dim,
            hidden_size=hid_dim,
            num_layers=n_layers,
            batch_first=True,
            dropout=dropout if n_layers > 1 else 0.0
        )

        # Output projection
        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):
        # input: (batch,) -> (batch, 1)
        input = input.unsqueeze(1)

        # Embed
        embedded = self.dropout(self.embedding(input))  # (batch, 1, emb_dim)

        # RNN step
        output, hidden = self.rnn(embedded, hidden)  # output: (batch, 1, hid_dim)

        # Predict logits
        prediction = self.fc_out(output.squeeze(1))  # (batch, output_dim)

        return prediction, hidden

### Task 3: The Seq2Seq Wrapper (30 Points)

This class combines the Encoder and Decoder and manages the overall process, including the decoding loop and Teacher Forcing.

**Instructions:**

1. Run the encoder on the source sequence and lengths to get the context vector (hidden).
2. Initialize the decoder input with the `<SOS>` token.
3. Iterate over the length of the target sequence:
    - Run the decoder one step.
    - Store the output.
    - Decide whether to use teacher forcing or the model's own prediction as the next input.

In [11]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, src_lengths, tgt, teacher_forcing_ratio=0.5):
        # src: (batch, src_len) | tgt: (batch, tgt_len)
        batch_size = src.size(0)
        tgt_len = tgt.size(1)
        tgt_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size, device=self.device)

        # 1) Encode
        hidden = self.encoder(src, src_lengths)  # (n_layers, batch, hid_dim)

        # 2) First decoder input is <SOS>
        input = torch.full((batch_size,), SOS_IDX, dtype=torch.long, device=self.device)

        # 3) Decode step-by-step
        for t in range(tgt_len):
            output, hidden = self.decoder(input, hidden)  # output: (batch, vocab)
            outputs[:, t, :] = output

            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)

            # 6) Next input
            input = tgt[:, t] if teacher_force else top1

        return outputs

---

## Part 5: Training the Model

### 5.1 Initialization (Provided)

We initialize the model with sensible hyperparameters. We use a relatively small model (2 layers, 512 hidden units) which provides a good balance of capacity and training speed for this dataset.

In [12]:
# Hyperparameters
INPUT_DIM = input_lang.n_words
OUTPUT_DIM = output_lang.n_words
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2 # Using 2 layers
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# Initialize models (Ensure Tasks 1-3 are completed first!)
enc = EncoderGRU(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT).to(device)
dec = DecoderGRU(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT).to(device)
model = Seq2Seq(enc, dec, device).to(device)

# Initialize weights (common practice for RNNs)
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
model.apply(init_weights)

# Optimizer
optimizer = optim.Adam(model.parameters())

# Loss function: CrossEntropyLoss, ignoring the padding index
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,162,154 trainable parameters


### Task 4: The Training and Evaluation Loops (20 Points)

Implement the training and evaluation functions.

**Instructions:**

1. In `train`, implement the forward pass, loss calculation, backpropagation (BPTT), gradient clipping, and optimizer step.
2. In `evaluate`, implement the forward pass (with `teacher_forcing_ratio=0`).
3. **Crucial:** Reshape the output and tgt tensors correctly for the loss function. CrossEntropyLoss expects predictions of shape `(N, C)` and targets of shape `(N)`, where N is the total number of tokens.

In [15]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        src, src_lengths, tgt = batch
        optimizer.zero_grad()

        # 1) Forward (default teacher forcing)
        output = model(src, src_lengths, tgt)

        # 2) Flatten for CE loss
        output_dim = output.shape[-1]
        output = output.reshape(-1, output_dim)
        tgt = tgt.reshape(-1)

        # 3) Loss
        loss = criterion(output, tgt)

        # 4) Backprop
        loss.backward()

        # 5) Clip
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # 6) Step
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src, src_lengths, tgt = batch

            # 1) Forward (no teacher forcing)
            output = model(src, src_lengths, tgt, teacher_forcing_ratio=0.0)

            # 2) Flatten
            output_dim = output.shape[-1]
            output = output.reshape(-1, output_dim)
            tgt = tgt.reshape(-1)

            # 3) Loss
            loss = criterion(output, tgt)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

### 5.3 Running the Training (Provided)

In [18]:
N_EPOCHS = 30
CLIP = 1.0

best_valid_loss = float('inf')
print("Starting training...")

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, val_loader, criterion)

    end_time = time.time()
    epoch_secs = int(end_time - start_time)

    # Save best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq-gru-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Starting training...
Epoch: 01 | Time: 4s
	Train Loss: 4.912 | Train PPL: 135.850
	 Val. Loss: 4.508 |  Val. PPL:  90.725
Epoch: 02 | Time: 2s
	Train Loss: 4.059 | Train PPL:  57.940
	 Val. Loss: 4.022 |  Val. PPL:  55.810
Epoch: 03 | Time: 2s
	Train Loss: 3.505 | Train PPL:  33.274
	 Val. Loss: 3.683 |  Val. PPL:  39.754
Epoch: 04 | Time: 2s
	Train Loss: 3.103 | Train PPL:  22.275
	 Val. Loss: 3.466 |  Val. PPL:  32.021
Epoch: 05 | Time: 2s
	Train Loss: 2.768 | Train PPL:  15.919
	 Val. Loss: 3.260 |  Val. PPL:  26.040
Epoch: 06 | Time: 2s
	Train Loss: 2.486 | Train PPL:  12.011
	 Val. Loss: 3.112 |  Val. PPL:  22.458
Epoch: 07 | Time: 2s
	Train Loss: 2.252 | Train PPL:   9.507
	 Val. Loss: 2.992 |  Val. PPL:  19.930
Epoch: 08 | Time: 2s
	Train Loss: 2.057 | Train PPL:   7.824
	 Val. Loss: 2.936 |  Val. PPL:  18.831
Epoch: 09 | Time: 2s
	Train Loss: 1.895 | Train PPL:   6.649
	 Val. Loss: 2.881 |  Val. PPL:  17.825
Epoch: 10 | Time: 2s
	Train Loss: 1.746 | Train PPL:   5.734
	 Val. Lo

**Important Note on Training Duration:**

Vanilla Seq2Seq models typically require 30-50+ epochs to produce reasonable translations. If you train for only 15 epochs, you will likely see:
- Decreasing loss (showing the model is learning)
- But poor actual translations (the model hasn't converged yet)

This is normal! The model needs more time to learn the complex mapping between languages.

---

## Part 6: Inference and Analysis (10 Points)

### 6.1 Inference (Greedy Decoding)

During inference, we don't have the target sentence, so teacher forcing is impossible. We use the model's own predictions at each step. The simplest method is **Greedy Decoding**: always choose the word with the highest probability.

In [21]:
def translate_sentence(sentence, src_lang, tgt_lang, model, device, max_len=50):
    model.eval()

    # 1. Preprocess the input sentence (normalize and reverse!)
    normalized_sentence = normalizeString(sentence)
    reversed_sentence = ' '.join(normalized_sentence.split(' ')[::-1])

    # 2. Convert to indices and tensor
    indices = [src_lang.word2index.get(word, UNK_IDX) for word in reversed_sentence.split(' ')] + [EOS_IDX]
    src_tensor = torch.tensor(indices, dtype=torch.long).unsqueeze(0).to(device) # (1, T)
    src_len = torch.tensor([len(indices)])

    # 3. Encode the sentence
    with torch.no_grad():
        hidden = model.encoder(src_tensor, src_len)

    # 4. Start decoding
    trg_indices = [SOS_IDX]
    input_tensor = torch.tensor([SOS_IDX], dtype=torch.long).to(device) # (1)

    for i in range(max_len):
        with torch.no_grad():
            output, hidden = model.decoder(input_tensor, hidden)

        # 5. Greedy Decoding
        pred_token = output.argmax(1).item()
        trg_indices.append(pred_token)

        # Check for <EOS>
        if pred_token == EOS_IDX:
            break

        # Prepare the next input
        input_tensor = torch.tensor([pred_token], dtype=torch.long).to(device)

    # 6. Convert indices back to words
    trg_tokens = [tgt_lang.index2word[i] for i in trg_indices]
    return trg_tokens[1:-1] # Exclude <SOS> and <EOS>

# Qualitative Analysis (Uncomment after training)
model.load_state_dict(torch.load('seq2seq-gru-model.pt'))
examples = ["i am cold", "she is happy", "he is running", "we are ready"]
for example in examples:
    translation = translate_sentence(example, input_lang, output_lang, model, device)
    print(f"EN: {example}")
    print(f"FR: {' '.join(translation)}\n")

EN: i am cold
FR: je je ?

EN: she is happy
FR: elle est en .

EN: he is running
FR: il est en .

EN: we are ready
FR: nous sommes en .



### 6.2 Understanding Your Results

**Expected Performance:**

After training, you may notice that your translations are not perfect - and that's completely normal! Here's what you should expect:

**What Good Results Look Like:**
- Training loss decreasing from ~5.0 to ~1.0-1.5
- Validation loss around 2.5-3.5
- Some simple phrases translating correctly (e.g., "how are you" → "comment vas tu")
- Shorter sentences working better than longer ones

**Why Translations May Be Poor:**

1. **The Information Bottleneck**: This is the fundamental limitation we've been discussing. The entire English sentence must be compressed into a single fixed-size vector (512 numbers). For complex sentences, critical information gets lost.

2. **Insufficient Training**: 30 epochs on this small dataset is barely enough. Production NMT systems train for much longer on millions of examples.

3. **Overfitting**: If your validation loss is significantly higher than training loss (e.g., 2.8 vs 1.3), the model is memorizing training patterns rather than learning to translate.

4. **Common Phrase Bias**: The model often outputs frequent French phrases (like "je suis...") regardless of the actual input, because these patterns were common in training data.

5. **Greedy Decoding**: We always pick the highest probability word. Beam search (which considers multiple possibilities) would improve results.

**What Your Model Is Actually Learning:**

Look at a translation like:
```
"i am cold" → "je suis serieux"
```

The model correctly learned:
- "i am" → "je suis" ✓
- But outputs a common word "serieux" instead of "froid"

This shows the model IS learning French grammar and common patterns, just not the specific vocabulary mapping yet.

**This Is Why Attention Was Invented!**

The poor performance of vanilla Seq2Seq on longer sentences directly motivated the invention of attention mechanisms (covered in the next module). Attention allows the decoder to "look back" at different parts of the input instead of relying on a single compressed vector.

---

### 6.3 Bonus: Diagnostic Function (Optional)

To better understand what your model has learned, implement this diagnostic function that checks if the model can at least memorize some training examples:

In [22]:
def diagnose_model(model, src_lang, tgt_lang, pairs, device, num_examples=5):
    """
    Check if model can translate training examples (memorization test)
    """
    print("\n" + "=" * 70)
    print("MODEL DIAGNOSIS - Testing on Training Examples")
    print("=" * 70)
    
    for i in range(num_examples):
        en_sentence = pairs[i][0]
        fr_actual = pairs[i][1]
        fr_predicted = translate_sentence(en_sentence, src_lang, tgt_lang, model, device)
        
        print(f"\nExample {i+1}:")
        print(f"  EN (input):     {en_sentence}")
        print(f"  FR (expected):  {fr_actual}")
        print(f"  FR (predicted): {' '.join(fr_predicted)}")
        
        # Calculate word overlap
        expected_words = set(fr_actual.split())
        predicted_words = set(fr_predicted)
        overlap = expected_words.intersection(predicted_words)
        if len(expected_words) > 0:
            accuracy = len(overlap) / len(expected_words) * 100
            print(f"  Word overlap:   {len(overlap)}/{len(expected_words)} ({accuracy:.1f}%)")

# Run after loading best model
diagnose_model(model, input_lang, output_lang, pairs, device)


MODEL DIAGNOSIS - Testing on Training Examples

Example 1:
  EN (input):     go .
  FR (expected):  va !
  FR (predicted): va !
  Word overlap:   2/2 (100.0%)

Example 2:
  EN (input):     run !
  FR (expected):  cours !
  FR (predicted): courez !
  Word overlap:   1/2 (50.0%)

Example 3:
  EN (input):     run !
  FR (expected):  courez !
  FR (predicted): courez !
  Word overlap:   2/2 (100.0%)

Example 4:
  EN (input):     wow !
  FR (expected):  ca alors !
  FR (predicted): ca alors !
  Word overlap:   3/3 (100.0%)

Example 5:
  EN (input):     fire !
  FR (expected):  au feu !
  FR (predicted): au feu !
  Word overlap:   3/3 (100.0%)


If the model can't even memorize training examples with >50% word overlap, it needs more training epochs or there may be a bug.

---

### 6.4 Conceptual Questions

Answer the following questions in a separate text cell or document:

1. **The Information Bottleneck**: The core limitation of this architecture is that the encoder must compress the entire input sentence into a single fixed-size context vector (hidden). Why is this a significant problem when translating very long or complex sentences?

This architecture forces the encoder to compress all information about the input sentence—its syntax, semantics, and context—into a single fixed-length vector. For short sentences, this can be adequate, but for long or complex ones, important details are lost because the hidden state cannot represent all dependencies effectively. As a result, the decoder lacks access to earlier parts of the input, leading to incomplete or incorrect translations.

2. **Input Reversal**: Explain again, in your own words, why reversing the input (the "Reversal Trick") helped the model learn more effectively. Relate your answer to the concept of gradient flow in BPTT.

Reversing the input shortens the effective distance between corresponding words in the source and target sequences. During Backpropagation Through Time (BPTT), gradients must flow through fewer time steps to connect related words, reducing vanishing gradient effects. This helps the model learn word alignments more easily and speeds up convergence in training.

3. **TBPTT Application**: While we used standard BPTT here, describe a different NLP task where Truncated BPTT (TBPTT) would be essential, and explain why standard BPTT would be unsuitable in that scenario.

Truncated BPTT would be essential in tasks such as language modeling on long documents or streaming text, where sequences can have thousands of tokens. Standard BPTT would require storing all intermediate activations across the full sequence, which is computationally infeasible. TBPTT limits backpropagation to manageable chunks, allowing the model to train efficiently while still maintaining continuity through hidden states.

4. **Packing**: Why is it important to use `pack_padded_sequence` in the encoder when dealing with batched inputs? What might happen if we didn't use it?

`pack_padded_sequence` ensures the RNN ignores padded elements when processing batched sequences of different lengths. Without packing, the model would treat padding tokens as real inputs, causing gradients and hidden states to be influenced by meaningless padding values. This would waste computation and degrade both training efficiency and accuracy.

For those that are interested to improve the performance, try to add:[optional]
- Beam Search for better decoding (instead of greedy)
- Better evaluation metrics (BLEU score)

In [29]:
# Optional Extension: Beam Search + BLEU Evaluation

import numpy as np
import torch
import torch.nn.functional as F
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smooth = SmoothingFunction().method1

def translate_beam(sentence, src_lang, tgt_lang, model, device, max_len=50, beam_size=5):
    """Simple beam search (no length norm)."""
    model.eval()

    # preprocess (normalize + reverse)
    norm = normalizeString(sentence)
    rev = ' '.join(norm.split(' ')[::-1])
    idxs = [src_lang.word2index.get(w, UNK_IDX) for w in rev.split(' ')] + [EOS_IDX]
    src = torch.tensor(idxs, dtype=torch.long).unsqueeze(0).to(device)  # (1, T)
    src_len = torch.tensor([len(idxs)])

    with torch.no_grad():
        hidden = model.encoder(src, src_len)

    # beams: list of (token_seq_tensor, score, hidden)
    beams = [(torch.tensor([SOS_IDX], device=device), 0.0, hidden)]
    finished = []

    for _ in range(max_len):
        new_beams = []
        for seq, score, h in beams:
            inp = seq[-1].unsqueeze(0)  # (1,)
            with torch.no_grad():
                out, h2 = model.decoder(inp, h)  # out: (1, vocab)
            logp = F.log_softmax(out, dim=1).squeeze(0)  # (vocab,)
            topk = torch.topk(logp, beam_size)
            for tok, tok_lp in zip(topk.indices, topk.values):
                tok = tok.item()
                new_seq = torch.cat([seq, torch.tensor([tok], device=device)])
                new_score = score + tok_lp.item()
                if tok == EOS_IDX:
                    finished.append((new_seq, new_score))
                else:
                    new_beams.append((new_seq, new_score, h2))
        beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_size]
        if not beams:
            break

    if not finished:
        finished = beams
    best_seq = max(finished, key=lambda x: x[1])[0].tolist()

    # map tokens to words, drop SOS/EOS/PAD
    toks = [tgt_lang.index2word[i] for i in best_seq[1:-1] if i not in (SOS_IDX, EOS_IDX, PAD_IDX)]
    return toks

def bleu_on_examples(model, pairs, src_lang, tgt_lang, device, n=20, beam_size=5, seed=123):
    """Compute sentence-level BLEU on n random pairs using beam decoding."""
    random.seed(seed)
    subset = random.sample(pairs, min(n, len(pairs)))
    scores = []

    for i, (en_sentence, fr_ref) in enumerate(subset, 1):
        pred_tokens = translate_beam(en_sentence, src_lang, tgt_lang, model, device, beam_size=beam_size)
        ref_tokens = fr_ref.split()
        bleu = sentence_bleu([ref_tokens], pred_tokens, smoothing_function=smooth)
        scores.append(bleu)
        print(f"[{i:02d}] EN: {en_sentence}")
        print(f"     REF: {' '.join(ref_tokens)}")
        print(f"     PRD: {' '.join(pred_tokens)}")
        print(f"     BLEU: {bleu:.3f}\n")

    avg_bleu = float(np.mean(scores)) if scores else 0.0
    print(f"Average BLEU over {len(scores)} examples: {avg_bleu:.3f}")
    return scores, avg_bleu

# Example usage
model.load_state_dict(torch.load('seq2seq-gru-model.pt', map_location=device))
_ = bleu_on_examples(model, pairs, input_lang, output_lang, device, n=10, beam_size=5)

[01] EN: be serious .
     REF: soyez serieuses !
     PRD: sois serieuse !
     BLEU: 0.114

[02] EN: how thrilling !
     REF: comme c est excitant !
     PRD: comme c est romantique !
     BLEU: 0.286

[03] EN: what is it ?
     REF: qu est ce ?
     PRD: qu est ce que c est ?
     BLEU: 0.176

[04] EN: i was unprepared .
     REF: je n etais pas prepare .
     PRD: je n ai pas arme .
     BLEU: 0.103

[05] EN: i have no idea .
     REF: aucune idee .
     PRD: je n ai aucune idee .
     BLEU: 0.202

[06] EN: here he comes .
     REF: le voila .
     PRD: le voila .
     BLEU: 0.562

[07] EN: i feel weak .
     REF: je me sens faible .
     PRD: je me sens faible .
     BLEU: 1.000

[08] EN: this feels right .
     REF: ca semble correct .
     PRD: ca semble correct .
     BLEU: 1.000

[09] EN: are you from here ?
     REF: vous etes du coin ?
     PRD: vous habitez ici ?
     BLEU: 0.074

[10] EN: who painted that ?
     REF: qui a peint cela ?
     PRD: qui a peint ceci ?
     BL

In [30]:
def translate_greedy(sentence, src_lang, tgt_lang, model, device, max_len=50):
    return translate_sentence(sentence, src_lang, tgt_lang, model, device, max_len=max_len)

def compare_decoders(model, pairs, src_lang, tgt_lang, device, n=10, seed=123):
    random.seed(seed)
    subset = random.sample(pairs, min(n, len(pairs)))
    for i, (en_sentence, fr_ref) in enumerate(subset, 1):
        pred_g = translate_greedy(en_sentence, src_lang, tgt_lang, model, device)
        pred_b = translate_beam(en_sentence, src_lang, tgt_lang, model, device, beam_size=5)
        ref = fr_ref.split()
        bleu_g = sentence_bleu([ref], pred_g, smoothing_function=smooth)
        bleu_b = sentence_bleu([ref], pred_b, smoothing_function=smooth)
        print(f"[{i:02d}] EN: {en_sentence}")
        print(f"     REF: {' '.join(ref)}")
        print(f"   GREEDY: {' '.join(pred_g)} | BLEU: {bleu_g:.3f}")
        print(f"     BEAM: {' '.join(pred_b)} | BLEU: {bleu_b:.3f}\n")

compare_decoders(model, pairs, input_lang, output_lang, device, n=8)

[01] EN: be serious .
     REF: soyez serieuses !
   GREEDY: sois serieuse ! | BLEU: 0.114
     BEAM: sois serieuse ! | BLEU: 0.114

[02] EN: how thrilling !
     REF: comme c est excitant !
   GREEDY: comme c est romantique ! | BLEU: 0.286
     BEAM: comme c est romantique ! | BLEU: 0.286

[03] EN: what is it ?
     REF: qu est ce ?
   GREEDY: qu est ce que c est ? | BLEU: 0.176
     BEAM: qu est ce que c est ? | BLEU: 0.176

[04] EN: i was unprepared .
     REF: je n etais pas prepare .
   GREEDY: je n ai pas arme . | BLEU: 0.103
     BEAM: je n ai pas arme . | BLEU: 0.103

[05] EN: i have no idea .
     REF: aucune idee .
   GREEDY: je n ai aucune idee . | BLEU: 0.202
     BEAM: je n ai aucune idee . | BLEU: 0.202

[06] EN: here he comes .
     REF: le voila .
   GREEDY: le voila . | BLEU: 0.562
     BEAM: le voila . | BLEU: 0.562

[07] EN: i feel weak .
     REF: je me sens faible .
   GREEDY: je me sens faible . | BLEU: 1.000
     BEAM: je me sens faible . | BLEU: 1.000

[08] EN: 